In [36]:
import urllib.request
from bs4 import BeautifulSoup
companies = ['facebook','alibaba']

for company in companies:
    quote_page = quote_page.replace("$REPLACE", company)
    page = urllib.request.urlopen(quote_page)
    soup = BeautifulSoup(page, "html.parser")

    #look for aria-level="2"
    arias = soup.find_all('a')
    links = []
    for aria in arias:
        if aria.get('href')[0:1]=="h":
            links.append(aria.get('href'))

    links = list(dict.fromkeys(links))
    for link in links[22:30]:
        print(link)

https://www.cnbc.com/2018/02/12/concerns-over-alibaba-led-digital-free-trade-zone-in-malaysia.html
https://www.malaysiakini.com/news/412130
https://www.fool.com/investing/2018/02/13/the-5-most-important-figures-from-alibabas-q3-repo.aspx
https://www.retaildive.com/news/alibaba-throws-13b-into-brick-and-mortar-strategy/516937/
https://www.theverge.com/2018/2/12/17003332/alibaba-disney-animation-cartoons-movies-youku
https://techcrunch.com/2018/02/12/alibaba-signs-deal-with-disney-to-stream-its-animated-shows-in-china/
http://variety.com/2018/digital/asia/disney-cartoons-in-china-on-alibaba-youku-platform-1202694948/
https://www.fool.com/investing/2018/02/12/3-things-alibaba-management-wants-you-to-know.aspx
